# Iterated Prisoner's Dilemma On A Network

## Imports and Config

In [456]:
import os
import math
import random
import logging
import numpy as np
import pandas as pd
import networkx as nx
from functools import partial
from dataclasses import dataclass
from IPython.display import display
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
from matplotlib.ticker import PercentFormatter
from matplotlib.animation import FuncAnimation


In [457]:
%matplotlib inline
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams["animation.embed_limit"] = 500

In [458]:
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(name)s - %(message)s",
)

## Matrices

In [459]:
payoff_matrices = {  # some of these don't look right
    "Default": {
        ("C", "C"): (3, 3),
        ("C", "D"): (0, 5),
        ("D", "C"): (5, 0),
        ("D", "D"): (0, 0),
    },
    # "Canonical": {
    #     ("C", "C"): (-1, -1),
    #     ("C", "D"): (-3, 0),
    #     ("D", "C"): (0, -3),
    #     ("D", "D"): (-2, -2),
    # },
    # "Friend or Foe": {
    #     ("C", "C"): (1, 1),
    #     ("C", "D"): (0, 2),
    #     ("D", "C"): (2, 0),
    #     ("D", "D"): (0, 0),
    # },
    # "Snowdrift": {
    #     ("C", "C"): (500, 500),
    #     ("C", "D"): (200, 800),
    #     ("D", "C"): (800, 200),
    #     ("D", "D"): (0, 0),
    # },
    # "Prisoners": {
    #     ("C", "C"): (500, 500),
    #     ("C", "D"): (-200, 1200),
    #     ("D", "C"): (1200, -200),
    #     ("D", "D"): (0, 0),
    # },
}

## Classes

### Strategy

In [460]:
class ActionStrategy:
    """Strategy that always plays its current action, randomly initialized."""

    def __init__(self, rng):
        self.rng = rng
        self.action = "C" if self.rng.random() < 0.5 else "D"

    def decide(self, agent_history):
        return self.action

    def set_action(self, action):
        self.action = action

In [461]:
class ImitationStrategy(ActionStrategy):
    """Imitate the action with the highest mean payoff in interactions."""

    def decide(self, agent_history):
        totals = {"C": 0.0, "D": 0.0}
        counts = {"C": 0, "D": 0}
        for interactions in agent_history.values():
            for inter in interactions:
                counts[inter.own_action] += 1
                counts[inter.neighbor_action] += 1
                totals[inter.own_action] += inter.own_reward
                totals[inter.neighbor_action] += inter.neighbor_reward

        mean_C = totals["C"] / counts["C"] if counts["C"] else 0
        mean_D = totals["D"] / counts["D"] if counts["D"] else 0

        if mean_C > mean_D:
            self.action = "C"
        elif mean_D > mean_C:
            self.action = "D"
        else:  # in case of a tie, continue with the existing strategy
            pass

        return self.action

In [462]:
class FermiStrategy(ActionStrategy):
    """
    Endogenous Fermi algorithm (pairwise).
    """

    def __init__(self, rng, temperature=0.1):
        super().__init__(rng)
        self.K = temperature

    def decide(self, agent_history):
        # if no neighbours yet, keep current action
        if not agent_history:
            return self.action

        # pick one random neighbour we've interacted with
        neighbour_id = self.rng.choice(list(agent_history.keys()))
        interactions = agent_history.get(neighbour_id, [])
        if not interactions:
            return self.action

        # estimate payoffs from history with that neighbour
        own_rewards = [i.own_reward for i in interactions]
        neigh_rewards = [i.neighbor_reward for i in interactions]

        payoff_self = float(np.mean(own_rewards)) if own_rewards else 0.0
        payoff_neigh = float(np.mean(neigh_rewards)) if neigh_rewards else 0.0

        delta = payoff_neigh - payoff_self

        # Fermi probability
        if self.K == 0:
            p_switch = 1.0 if delta > 0 else 0.0
        else:
            exponent = -delta / self.K
            exponent = max(min(exponent, 700), -700)  # avoid overflow
            p_switch = 1.0 / (1.0 + math.exp(exponent))

        # switch action to neighbour's most recently observed action
        if self.rng.random() < p_switch:
            self.action = interactions[-1].neighbor_action

        return self.action

In [463]:
class ReinforcementLearningStrategy(ActionStrategy):
    """
    Q-learning strategy with epsilon-greedy action selection.
    """

    def __init__(self, rng, learning_rate=0.1, epsilon=0.1, initial_q=0.0):
        super().__init__(rng)
        self.alpha = learning_rate
        self.epsilon = epsilon
        self.q = {"C": float(initial_q), "D": float(initial_q)}
        self._last_action = None
        self._last_reward = 0.0

    def decide(self, agent_history):
        # observe most recent reward from any interaction (if exists)
        last = None
        for interactions in agent_history.values():
            if interactions:
                cand = interactions[-1]
                if last is None:
                    last = cand
        if last is not None:
            self._last_reward = last.own_reward

        # update Q for the action we previously played
        if self._last_action is not None:
            a = self._last_action
            self.q[a] = self.q[a] + self.alpha * (self._last_reward - self.q[a])

        # choose next action (epsilon-greedy)
        if self.rng.random() < self.epsilon:
            action = "C" if self.rng.random() < 0.5 else "D"
        else:
            if self.q["C"] > self.q["D"]:
                action = "C"
            elif self.q["D"] > self.q["C"]:
                action = "D"
            else:
                action = "C" if self.rng.random() < 0.5 else "D"

        self.action = action
        self._last_action = action
        return self.action

### Agent

In [464]:
class Agent:
    """Minimal agent holding a strategy, payoff, and history."""

    @dataclass
    class Interaction:
        own_action: str
        own_reward: float
        neighbor_action: str
        neighbor_reward: float

    def __init__(self, agent_id, strategy, history_window=5, store_history=True):
        self.id = agent_id
        self.strategy = strategy
        self.history = {}
        self.payoff = 0.0
        self.history_window = history_window
        self.store_history = store_history

    def choose_action(self):
        return self.strategy.decide(self.history)

    def record_interaction(
        self, neighbor_id, own_action, neighbor_action, own_reward, neighbor_reward
    ):
        self.payoff += own_reward
        if not self.store_history:
            return
        lst = self.history.setdefault(neighbor_id, [])
        lst.append(
            self.Interaction(own_action, own_reward, neighbor_action, neighbor_reward)
        )
        if len(lst) > self.history_window:
            del lst[: -self.history_window]

In [465]:

# Jaccard Index for stability
def jaccard_indices(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union > 0 else 0

# Cluster sizes
def cluster_sizes(graph, cooperators):
    subgraph = graph.subgraph(cooperators)
    return [len(c) for c in nx.connected_components(subgraph)]

# Largest cooperative fraction
def largest_cooperative_fraction(graph, cooperators):
    subgraph = graph.subgraph(cooperators)
    largest_cluster = max(nx.connected_components(subgraph), key=len, default=[])
    return len(largest_cluster) / graph.number_of_nodes()

# Stability of largest cooperative cluster
def stability(largest_cluster_prev, largest_cluster_curr):
    if not largest_cluster_prev or not largest_cluster_curr:
        return 0.0
    return len(largest_cluster_prev & largest_cluster_curr) / len(largest_cluster_curr)


# Average time nodes spend in the largest cluster
def node_times_in_cluster(node_times, current_cluster):
    for node in current_cluster:
        node_times[node] = node_times.get(node, 0) + 1
    return sum(node_times.values()) / len(node_times) if node_times else 0

## Helpers

In [466]:
def smooth(x, window=20):
    x = np.asarray(x)
    if len(x) < window:
        return x
    return np.convolve(x, np.ones(window)/window, mode="valid")


In [467]:
def jaccard_sets(A, B):
    if not A and not B:
        return 1.0
    if not A or not B:
        return 0.0
    return len(A & B) / len(A | B)


In [468]:
def boundary_length(graph, cooperators):
    count = 0
    for u, v in graph.edges():
        if (u in cooperators) != (v in cooperators):
            count += 1
    return count


In [469]:
def save_plot(fig, filename, output_dir="plots"):
    os.makedirs(output_dir, exist_ok=True)
    filepath = os.path.join(output_dir, filename)
    fig.savefig(filepath)
    plt.close(fig)

In [470]:
def collect_cluster_sizes(results, Tburn, strategy_label):
    sizes = []
    for r in results:
        for step_sizes in r[strategy_label][Tburn:]:
            sizes.extend(step_sizes)
    return sizes

In [471]:
def aggregate_runs(results, Tburn=0):
    keys = ["Smax", "c", "JV", "JL", "Neff", "B"]
    L = min(len(r[k]) for r in results for k in keys)
    out = {}
    for k in keys:
        X = np.stack([r[k][:L] for r in results])
        out[k] = {
            "mean_ts": X.mean(axis=0),
            "std_ts": X.std(axis=0, ddof=1),
            "mean_scalar": X[:, Tburn:].mean(),
        }
    return out

In [472]:
def collect_cluster_sizes_window(results, t_start, t_end):
    sizes = []
    for r in results:
        for step_sizes in r["sizes"][t_start:t_end]:
            sizes.extend(step_sizes)
    return sizes


In [473]:
def collect_Smax_vs_c(results, Tburn):
    xs, ys = [], []
    for r in results:
        c = r["c"][Tburn:]
        Smax = r["Smax"][Tburn:]
        xs.extend(c)
        ys.extend(Smax)
    return np.array(xs), np.array(ys)


### Network

In [474]:
class Network:
    """NetworkX graph wrapper."""

    def generate_graph(self, kind, n, seed=None, **kwargs):
        """Generate a networkx graph by name."""
        if kind == "grid":
            side_length = int(math.sqrt(n))
            graph = nx.convert_node_labels_to_integers(
                nx.grid_2d_graph(side_length, side_length)
            )

        elif kind == "stochastic_block":
            sizes = kwargs.pop("sizes")
            p = kwargs.pop("p")
            graph = nx.stochastic_block_model(sizes, p, seed=seed, **kwargs)

        else:
            generators = {
                "erdos_renyi": nx.erdos_renyi_graph,
                "watts_strogatz": nx.watts_strogatz_graph,
                "barabasi_albert": nx.barabasi_albert_graph,
            }
            graph = generators[kind](n, seed=seed, **kwargs)

        self.kind = kind
        self.graph = graph
        self.seed = seed

    def neighbour(self, agent_id):
        """Return neighbour agent IDs."""
        if self.graph:
            return list(self.graph.neighbors(agent_id))
        else:
            raise NotImplementedError

In [475]:
class NetworkSimulation(Network):
    """
    Base class for running evolutionary games on any NetworkX graph.
    """

    def __init__(
        self,
        kind="grid",
        n=400,
        seed=42,
        rounds=100,
        strategy=ActionStrategy,
        strategy_kwargs=None,
        payoff_matrix=payoff_matrices["Default"],
        rng=None,
        history_window=5,
        store_history=True,
        store_snapshots=True,
        **graph_kwargs,
    ):
        self.strategy = strategy
        self.strategy_kwargs = strategy_kwargs or {}
        self.rounds = rounds
        self.payoff_matrix = payoff_matrix
        self.rng = rng if rng is not None else np.random.default_rng(seed)
        self.history_window = history_window
        self.store_history = store_history
        self.store_snapshots = store_snapshots
        self.generate_graph(kind=kind, n=n, seed=seed, **graph_kwargs)
        print(nx.degree_histogram(self.graph))
        print(nx.average_clustering(self.graph))
        self.edge_list = list(self.graph.edges())
        self.agents = {}
        self.snapshots = []
        self._initialize_agents()
        self.jaccard_indices = []
        self.cluster_sizes = []
        self.num_components = []
        self.set_stability = []

        self.largest_cooperative_fraction = []
        self.stability = []
        self.cooperation_fraction = []
        self.node_times_in_cluster = {}
        self.prev_largest_cluster = set()
        self.prev_cooperators = set()
        self.effective_num_components = []
        self.boundary_per_node = []
        self.defector_cluster_sizes = []




    def _initialize_agents(self):
        for agent_id in self.graph.nodes:
            strat = self.strategy(self.rng, **self.strategy_kwargs)
            self.agents[agent_id] = Agent(
                agent_id,
                strat,
                history_window=self.history_window,
                store_history=self.store_history,
            )

    def _reset_payoffs(self):
        for agent in self.agents.values():
            agent.payoff = 0.0

    # fast inner loop: precompute lookups and actions once
    def _play_round(self):
        agents = self.agents
        payoff_matrix = self.payoff_matrix
        edge_list = self.edge_list

        for agent in agents.values():
            agent.payoff = 0.0

        actions = {node: agents[node].choose_action() for node in agents}
        for node_a, node_b in edge_list:
            action_a = actions[node_a]
            action_b = actions[node_b]
            payoff_a, payoff_b = payoff_matrix.get((action_a, action_b), (0, 0))
            agents[node_a].record_interaction(
                node_b, action_a, action_b, payoff_a, payoff_b
            )
            agents[node_b].record_interaction(
                node_a, action_b, action_a, payoff_b, payoff_a
            )
    
    def _update_cluster_metrics(self):
        cooperators = {
            node for node, agent in self.agents.items()
            if agent.strategy.action == "C"
        }
        
        c_frac = len(cooperators) / self.graph.number_of_nodes()
        self.cooperation_fraction.append(c_frac)
        sizes = cluster_sizes(self.graph, cooperators)
        self.cluster_sizes.append(sizes)
        subgraph = self.graph.subgraph(cooperators)
        largest_cluster = set(
            max(nx.connected_components(subgraph), key=len, default=[])
        )
        num_components = len(sizes)
        self.num_components.append(num_components)
        if self.prev_largest_cluster:
            jac = jaccard_indices(self.prev_largest_cluster, largest_cluster)
        else:
            jac = 0.0
        self.jaccard_indices.append(jac)
        stab = stability(self.prev_largest_cluster, largest_cluster)
        self.stability.append(stab)
        frac = len(largest_cluster) / self.graph.number_of_nodes()
        self.largest_cooperative_fraction.append(frac)
        avg_time = node_times_in_cluster(
            self.node_times_in_cluster, largest_cluster
        )
        if hasattr(self, "prev_cooperators"):
            JV = jaccard_sets(self.prev_cooperators, cooperators)
        else:
            JV = 0.0
        self.set_stability.append(JV)
        self.prev_cooperators = cooperators
        if sizes:
            total = sum(sizes)
            Neff = (total ** 2) / sum(s * s for s in sizes)
        else:
            Neff = 0.0

        self.effective_num_components.append(Neff)
        B = boundary_length(self.graph, cooperators)
        if cooperators:
            B_norm = B / len(cooperators)
        else:
            B_norm = 0.0

        self.boundary_per_node.append(B_norm)
        defectors = set(self.graph.nodes()) - cooperators
        defector_sizes = cluster_sizes(self.graph, defectors)
        self.defector_cluster_sizes.append(defector_sizes)
        self.prev_largest_cluster = largest_cluster


    def _get_state(self):
        return {
            node_id: (1 if agent.strategy.action == "D" else 0)
            for node_id, agent in self.agents.items()
        }

    def encode_state(self):  # for speed
        arr = np.fromiter(
            (
                1 if self.agents[i].strategy.action == "D" else 0
                for i in self.graph.nodes()
            ),
            dtype=np.uint8,
            count=self.graph.number_of_nodes(),
        )
        return np.packbits(arr, bitorder="little").tobytes()

    def decode_state(self, packed):
        n = self.graph.number_of_nodes()
        bits = np.unpackbits(np.frombuffer(packed, dtype=np.uint8), bitorder="little")
        return bits[:n].astype(np.uint8)

    def step(self):
        self._play_round()
        if self.store_snapshots:
            self.snapshots.append(self._get_state())
        self._update_cluster_metrics()

    def run(self):
        for _ in range(self.rounds):
            self.step()

## Visualization

In [476]:
def experiment(
    model_class,
    strategy_class,
    strategy_kwargs=None,
    steps=100,
    seed=42,
    interval=300,
    payoff_matrix=payoff_matrices["Default"],
    title="",
    kind="grid",
    n=400,
    is_grid=False,
    **graph_kwargs,
):
    """
    Produce animations showing the network state over time.
    """
    payoff_matrix = payoff_matrix
    strategy_kwargs = strategy_kwargs or {}
    model = model_class(
        kind=kind,
        n=n,
        seed=seed,
        rounds=steps,
        payoff_matrix=payoff_matrix,
        strategy=strategy_class,
        strategy_kwargs=strategy_kwargs,
        **graph_kwargs,
    )

    graph = model.graph
    n_nodes = graph.number_of_nodes()

    C_COOP, C_DEFECT = "#40B0A6", "#FFBE6A"
    cmap = ListedColormap([C_COOP, C_DEFECT])
    fig, (ax_sim, ax_stats) = plt.subplots(
        2, 1, figsize=(7, 8), gridspec_kw={"height_ratios": [4, 1]}
    )

    # -------------------------
    # Stats plot (C% and D%)
    # -------------------------
    xs, ys_c, ys_d = [], [], []

    (line_c,) = ax_stats.plot([], [], lw=2, label="C")
    (line_d,) = ax_stats.plot([], [], lw=2, label="D")

    ax_stats.set_xlim(0, steps)
    ax_stats.set_ylim(0, 100)
    ax_stats.yaxis.set_major_formatter(PercentFormatter(xmax=100))
    ax_stats.set_ylabel("Population")
    ax_stats.grid(True, linestyle=":", alpha=0.4)
    ax_stats.legend(frameon=False, ncol=2, loc="upper right")

    # -------------------------
    # Simulation plot
    # -------------------------
    if is_grid:
        dim = int(math.isqrt(n_nodes))
        if dim * dim != n_nodes:
            raise ValueError(f"Grid mode needs square number of nodes, got {n_nodes}.")

        def state_as_grid():
            state = model._get_state()
            grid = [[0] * dim for _ in range(dim)]
            for node, val in state.items():
                grid[node // dim][node % dim] = val
            return grid

        sim_artist = ax_sim.imshow(state_as_grid(), cmap=cmap, vmin=0, vmax=1)
        ax_sim.set_xticks([])
        ax_sim.set_yticks([])

        def update_sim():
            sim_artist.set_data(state_as_grid())

    else:
        pos = nx.spring_layout(graph, seed=seed)
        nodelist = list(graph.nodes())
        nx.draw_networkx_edges(graph, pos, ax=ax_sim, alpha=0.3, edge_color="gray")
        state0 = model._get_state()
        sim_artist = nx.draw_networkx_nodes(
            graph,
            pos,
            nodelist=nodelist,
            node_color=[state0[i] for i in nodelist],
            cmap=cmap,
            vmin=0,
            vmax=1,
            node_size=80,
            edgecolors="gray",
            ax=ax_sim,
        )
        ax_sim.axis("off")

        def update_sim():
            state = model._get_state()
            sim_artist.set_array([state[i] for i in nodelist])

    # -------------------------
    # Animation update
    # -------------------------
    def update(frame):
        if frame > 0:
            model.step()

        ax_sim.set_title(f"{title} (Step {frame}/{steps})")

        update_sim()

        state = model._get_state()
        d = sum(state.values())
        c = n_nodes - d

        xs.append(frame)
        ys_c.append(100 * c / n_nodes)
        ys_d.append(100 * d / n_nodes)

        line_c.set_data(xs, ys_c)
        line_d.set_data(xs, ys_d)

        return sim_artist, line_c, line_d

    
    anim= FuncAnimation(
        fig,
        update,
        frames=steps + 1,
        interval=interval,
        blit=True,
        repeat=False,
    )



    return anim

## Helpers for plots


In [477]:
strategies = {
    # "ActionStrategy": ActionStrategy,
    "ImitationStrategy": ImitationStrategy,
    # "FermiStrategy": FermiStrategy,
    # "ReinforcementLearningStrategy": ReinforcementLearningStrategy,
}

In [478]:
def plot_time_series(data, key, ylabel, title, ylim=None, filename=None):
    fig, ax = plt.subplots(figsize=(6, 4))
    t = range(len(data[key]["mean_ts"]))
    ax.plot(t, data[key]["mean_ts"], lw=2)
    ax.fill_between(
        t,
        data[key]["mean_ts"] - data[key]["std_ts"],
        data[key]["mean_ts"] + data[key]["std_ts"],
        alpha=0.3,
    )
    ax.set_xlabel("Time step")
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    if ylim:
        ax.set_ylim(ylim)
    ax.grid(alpha=0.3)
    plt.tight_layout()
    if filename:
        save_plot(fig, filename)

In [479]:
def plot_histogram(data, xlabel, ylabel, title, filename=None):
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.hist(data, bins=30, log=True)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    plt.tight_layout()
    if filename:
        save_plot(fig, filename)

In [480]:
def run_once(seed, **kwargs):
    model = NetworkSimulation(seed=seed, **kwargs)
    model.run()
    return {
        "Smax": np.array(model.largest_cooperative_fraction),
        "c":    np.array(model.cooperation_fraction),
        "JV":   np.array(model.set_stability),
        "JL":   np.array(model.jaccard_indices),
        "Neff": np.array(model.effective_num_components),
        "B":    np.array(model.boundary_per_node),
        "sizes": model.cluster_sizes,
        "defector_sizes": model.defector_cluster_sizes,
    }


In [481]:
def plot_time_resolved_cluster_distributions(
    results,
    Tburn,
    strategy,
    matrix,
    filename=None,
):
    windows = {
        "Early": (Tburn, Tburn + 30),
        "Mid":   (Tburn + 50, Tburn + 80),
        "Late":  (Tburn + 110, Tburn + 150),
    }

    fig, ax = plt.subplots(figsize=(6, 4))

    for label, (t0, t1) in windows.items():
        sizes = collect_cluster_sizes_window(results, t0, t1)

        ax.hist(
            sizes,
            bins=30,
            density=True,
            histtype="step",
            log=True,
            label=label,
        )

    ax.set_xlabel("Cooperative cluster size $s$")
    ax.set_ylabel("Probability density (log scale)")
    ax.set_title(
        f"Time-resolved cooperative cluster distributions\n({strategy}, {matrix})"
    )
    ax.legend()
    ax.grid(alpha=0.3)
    plt.tight_layout()

    if filename:
        save_plot(fig, filename)




In [482]:
def plot_Smax_vs_cooperation(
    results,
    Tburn,
    strategy,
    matrix,
    filename=None,
):
    c_vals, Smax_vals = collect_Smax_vs_c(results, Tburn)

    fig, ax = plt.subplots(figsize=(5, 5))

    ax.scatter(
        c_vals,
        Smax_vals,
        alpha=0.2,
        s=10,
    )

    ax.set_xlabel("Overall cooperation fraction $|V_C|/N$")
    ax.set_ylabel("Largest cooperative cluster fraction $S_{\\max}$")
    ax.set_title(
        f"Cluster dominance vs cooperation\n({strategy}, {matrix})"
    )
    ax.grid(alpha=0.3)
    plt.tight_layout()

    if filename:
        save_plot(fig, filename)



In [483]:
def run_simulation():
    seeds = range(20)  
    Tburn = 50  

    for strategy_name, strategy_class in strategies.items():
        for matrix_name, payoff_matrix in payoff_matrices.items():
            results = [
                run_once(
                    seed=s,
                    kind="grid",
                    n=400,
                    rounds=200,
                    strategy=strategy_class,
                    payoff_matrix=payoff_matrix,
                )
                for s in seeds
            ]

            out = aggregate_runs(results, Tburn=Tburn)

            plot_time_series(
                out,
                "Smax",
                ylabel="Largest cooperative component fraction $S_{\\max}(t)$",
                title=f"Evolution of the Largest Cooperative Cluster ({strategy_name}, {matrix_name})",
                ylim=(0, 0.2),
                filename=f"{strategy_name}_{matrix_name}_Smax.png",
            )
            plot_time_series(
                out,
                "c",
                ylabel="Fraction of cooperators $|V_C(t)| / N$",
                title=f"Overall Level of Cooperation ({strategy_name}, {matrix_name})",
                ylim=(0, 0.6),
                filename=f"{strategy_name}_{matrix_name}_cooperation.png",
            )
            plot_time_series(
                out,
                "JV",
                ylabel="Jaccard index $J_V(t)$",
                title=f"Stability of the Cooperative Set ({strategy_name}, {matrix_name})",
                ylim=(0, 1),
                filename=f"{strategy_name}_{matrix_name}_JV.png",
            )
            plot_time_series(
                out,
                "JL",
                ylabel="Overlap of largest component $J_L(t)$",
                title=f"Temporal Stability of the Largest Cooperative Cluster ({strategy_name}, {matrix_name})",
                ylim=(0, 1),
                filename=f"{strategy_name}_{matrix_name}_JL.png",
            )
            plot_time_series(
                out,
                "Neff",
                ylabel="Effective number of cooperative components $N_{\\mathrm{eff}}(t)$",
                title=f"Fragmentation of Cooperative Clusters ({strategy_name}, {matrix_name})",
                filename=f"{strategy_name}_{matrix_name}_Neff.png",
            )


            coop_sizes = collect_cluster_sizes(results, Tburn, "sizes")
            def_sizes = collect_cluster_sizes(results, Tburn, "defector_sizes")

            plot_histogram(
                coop_sizes,
                xlabel="Cooperative cluster size $s$",
                ylabel="Frequency (log scale)",
                title=f"Distribution of Cooperative Cluster Sizes ({strategy_name}, {matrix_name})",
                filename=f"{strategy_name}_{matrix_name}_coop_sizes.png",
            )
            plot_histogram(
                def_sizes,
                xlabel="Defector cluster size $s$",
                ylabel="Frequency (log scale)",
                title=f"Distribution of Defector Cluster Sizes ({strategy_name}, {matrix_name})",
                filename=f"{strategy_name}_{matrix_name}_def_sizes.png",
            )

            plot_time_resolved_cluster_distributions(
                results=results,
                Tburn=Tburn,
                strategy=strategy_name,
                matrix=matrix_name,
                filename=f"{strategy_name}_{matrix_name}_time_resolved_clusters.png",

            )

            plot_Smax_vs_cooperation(
                results=results,
                Tburn=Tburn,
                strategy=strategy_name,
                matrix=matrix_name,
                filename=f"{strategy_name}_{matrix_name}_Smax_vs_cooperation.png",

            )




In [484]:
run_simulation()

[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
[0, 0, 4, 72, 324]
0.0
